# # China → US Spillover Analysis (Jupyter Notebook)
# Simple, well-commented notebook that tests whether Chinese indices help predict US index returns.
# Assumes your CSVs already have columns: Date, Close, Idxrtn (so no renaming required).


### Quick overview
# Steps:
# 1. Load weekly data (assume weekly or resample to weekly)
# 2. Compute weekly log returns
# 3. Method A: VAR + Granger causality (lead-lag testing)
# 4. Method B: Diebold-Yilmaz spillover index (FEVD-based)
# 5. Method C: Volatility spillover via GARCH + rolling correlation of standardized residuals
# 6. Method D: ML-approach: XGBoost
# 7. Method E: ML-approach: small feed-forward neural net

## Setup

In [ ]:
pip install pandas numpy matplotlib seaborn statsmodels arch xgboost tensorflow

In [ ]:
import warnings
warnings.filterwarnings('ignore')


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


from statsmodels.tsa.api import VAR
from statsmodels.tsa.stattools import grangercausalitytests
from arch import arch_model


from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score


import xgboost as xgb
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping


sns.set_style('whitegrid')
np.random.seed(42)

## Data Loading

In [ ]:
us_path = 'us_index.csv'
cn_path = 'cn_index.csv'


us = pd.read_csv(us_path, parse_dates=['Date']).sort_values('Date')
cn = pd.read_csv(cn_path, parse_dates=['Date']).sort_values('Date')


# Resample both series to weekly (Friday close). If your data is already weekly, this will simply keep it.
us_w = us.set_index('Date')['Close'].resample('W-FRI').last().ffill()
cn_w = cn.set_index('Date')['Close'].resample('W-FRI').last().ffill()


# Build DataFrame with log returns
data = pd.DataFrame({'US_Close': us_w, 'CN_Close': cn_w}).dropna()
# Log returns: simple and commonly used for VAR / GARCH
data['US_Ret'] = np.log(data['US_Close']).diff()
data['CN_Ret'] = np.log(data['CN_Close']).diff()
# drop the first NA after differencing
data = data.dropna().reset_index().rename(columns={'index': 'Date'})


print('Sample rows:')
print(data.head())


plt.figure(figsize=(10,4))
plt.plot(data['Date'], data['US_Ret'], label='US weekly log return')
plt.plot(data['Date'], data['CN_Ret'], label='CN weekly log return')
plt.legend()
plt.title('Weekly log returns (US vs China)')
plt.show()

# %%
# Prepare data and fit VAR
var_data = data[['US_Ret', 'CN_Ret']].set_index(data['Date'])
model = VAR(var_data)
res = model.fit(4)
print(res.summary())

## Methods

### Granger Causality

Idea (simple): fit a VAR on the two return series and run Granger causality tests.
If lagged CN returns help predict US returns (statistically), that is evidence of return spillover from CN to US.


Notes on interpretation:
- Granger causality is not proof of economic causation. It tests whether past values of one series contain predictive information about another.
- Use several lags. Here we use 4 weekly lags (~1 month).


In [ ]:
# Prepare data and fit VAR
var_data = data[['US_Ret', 'CN_Ret']].set_index(data['Date'])
model = VAR(var_data)
res = model.fit(4)
print(res.summary())

# The function expects an array where column 0 is the dependent variable
# and column 1 is the potential cause. We test both directions.
print('Granger test: does CN cause US? (null: no causality)')
grangercausalitytests(var_data[['US_Ret','CN_Ret']], maxlag=4, verbose=True)


print('Granger test: does US cause CN? (null: no causality)')
grangercausalitytests(var_data[['CN_Ret','US_Ret']], maxlag=4, verbose=True)

### Diebold-Yilmaz spillover index
Idea: use the VAR's forecast error variance decomposition (FEVD). The fraction of US forecast error variance
explained by shocks to CN (the off-diagonal element) is a direct measure of spillover from CN to US.


This is a simplified one-step FEVD approach. Robust research implementations use generalized FEVD and
bootstrap inference. This version is for learning and quick checks.


In [ ]:
h = 10 # FEVD horizon in periods (weeks)
fevd = res.fevd(h)
# fevd.decomp: shape (h, neqs, neqs) — take the last horizon
try:
decomp = fevd.decomp[-1]
except Exception:
# statsmodels versions differ; fall back to direct attribute
decomp = fevd._decomp[-1]


# normalize rows to sum to 1 (so we interpret as shares)
row_sums = decomp.sum(axis=1, keepdims=True)
shares = decomp / row_sums


# total spillover index: mean of off-diagonal shares
n = shares.shape[0]
off_diag = shares.sum() - np.trace(shares)
total_spill = off_diag / n
print(f'Total spillover index (h={h}): {total_spill:.4f}')


spill_df = pd.DataFrame(shares, index=['US_recipient','CN_recipient'], columns=['US_source','CN_source'])
print('FEVD share matrix (rows = recipient, cols = source):')
print(spill_df)

## 5. Method C: Volatility spillover (GARCH + rolling correlation of standardized residuals)
Idea: Fit univariate GARCH(1,1) to each return series to model conditional volatility.
Then compute standardized residuals (residual / cond. vol). If those standardized residuals show
time-varying correlation (e.g., rising after shocks), that is suggestive of volatility spillover / contagion.


This is a clear, easy-to-implement proxy. For full multivariate modeling use DCC-GARCH or BEKK.

In [ ]:
# Fit GARCH(1,1) to each returns series (scale returns because small numbers can cause numerical issues)
us_garch = arch_model(data['US_Ret']*100, p=1, q=1).fit(disp='off')
cn_garch = arch_model(data['CN_Ret']*100, p=1, q=1).fit(disp='off')


# standardized residuals
us_std = us_garch.resid / us_garch.conditional_volatility
cn_std = cn_garch.resid / cn_garch.conditional_volatility


# Rolling correlation of standardized residuals (window=52 weeks ~ 1 year)
roll_corr = us_std.rolling(window=52, min_periods=20).corr(cn_std)


plt.figure(figsize=(10,4))
# roll_corr aligns with the index of the original series (same length)
plt.plot(data['Date'], roll_corr)
plt.title('Rolling correlation of standardized residuals (window=52)')
plt.show()


print(f'Average rolling correlation: {roll_corr.mean():.4f}')

### Machine Learning

ML: Predict US returns using lagged China returns (XGBoost and small NN)
Build simple lag features and evaluate using time-series cross-validation to avoid look-ahead bias.

Rules used here:
- Use lagged CN returns (1..4) and optionally lagged US returns for autoregressive info
- Evaluate with TimeSeriesSplit, reporting RMSE and R^2

In [ ]:
# Lags

def make_lags(df, lags=4):
    # df must have columns Date, US_Ret, CN_Ret
    tmp = df.copy().set_index('Date')
    for i in range(1, lags+1):
    tmp[f'CN_lag{i}'] = tmp['CN_Ret'].shift(i)
    tmp[f'US_lag{i}'] = tmp['US_Ret'].shift(i)
    tmp = tmp.dropna().reset_index()
    return tmp


lags = 4
ml_df = make_lags(data, lags=lags)
features = [c for c in ml_df.columns if 'lag' in c]
X = ml_df[features].values
y = ml_df['US_Ret'].values


print('ML sample size:', X.shape)
print('Features:', features)

#### XGBoost
Simple regressor with default settings. TimeSeriesSplit prevents using future information.

In [ ]:
tscv = TimeSeriesSplit(n_splits=5)
preds_xgb = np.zeros_like(y)
for train_idx, test_idx in tscv.split(X):
X_train, X_test = X[train_idx], X[test_idx]
y_train, y_test = y[train_idx], y[test_idx]
model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, random_state=42)
model.fit(X_train, y_train)
preds_xgb[test_idx] = model.predict(X_test)


print('XGBoost RMSE:', np.sqrt(mean_squared_error(y, preds_xgb)))
print('XGBoost R2:', r2_score(y, preds_xgb))

#### Neural Network
Small fully connected network. We scale features and keep architecture tiny to avoid overfitting on small samples.

In [ ]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
preds_nn = np.zeros_like(y)


for train_idx, test_idx in tscv.split(X_scaled):
    X_train, X_test = X_scaled[train_idx], X_scaled[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]
    nn = Sequential([
        Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
        Dropout(0.2),
        Dense(32, activation='relu'),
        Dense(1)
    ])
    
nn.compile(optimizer='adam', loss='mse')
es = EarlyStopping(patience=10, restore_best_weights=True, verbose=0)
nn.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100, verbose=0, callbacks=[es])
preds_nn[test_idx] = nn.predict(X_test).flatten()


print('NN RMSE:', np.sqrt(mean_squared_error(y, preds_nn)))
print('NN R2:', r2_score(y, preds_nn))

#### ML methods evaluation

In [ ]:
# Plot ML predictions vs. actual

plt.figure(figsize=(12,5))
plt.plot(ml_df['Date'], y, label='Actual US_Ret', linewidth=1)
plt.plot(ml_df['Date'], preds_xgb, label='XGBoost', alpha=0.8)
plt.plot(ml_df['Date'], preds_nn, label='Neural Net', alpha=0.8)
plt.legend()
plt.title('ML predictions vs actual US weekly returns')
plt.show()